In [167]:
import pandas as pd
import numpy as np
import pandas_gbq
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV, KFold
# from io import BytesIO
from google.cloud import storage
import os

In [ ]:
# df = pd.read_csv('gs://car_platform_gcs_bucket/data/vehicles.csv', sep=",")

In [174]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./car-platform-273211-beffc197afb6.json"

In [12]:
query = """
SELECT
  *
FROM
  car-platform-273211.car_listings.car_listings
"""
df = pd.read_gbq(query,
                     project_id='car-platform-273211',
                     dialect='standard')

In [13]:
df.shape

(539759, 25)

In [14]:
df.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,drive,size,type,paint_color,image_url,description,county,state,lat,long
0,7090040033,https://sd.craigslist.org/cto/d/winner-2013-gm...,south dakota,https://sd.craigslist.org,20000,2013.0,gmc,acadia denali,excellent,None,...,None,None,None,red,https://images.craigslist.org/00A0A_qxTivbw44s...,"84,300 miles New tires, well maintained. Love...",None,sd,43.3318,-99.8033
1,7074852055,https://pueblo.craigslist.org/ctd/d/pueblo-201...,pueblo,https://pueblo.craigslist.org,5950,2016.0,kia,soul,None,None,...,fwd,None,SUV,black,https://images.craigslist.org/00q0q_2ArR8F6mYq...,"2016 KIA SOUL - $5,950 \tGallery Auto Auction ...",None,co,38.3536,-104.6230
2,7074738939,https://westslope.craigslist.org/ctd/d/grand-j...,western slope,https://westslope.craigslist.org,4900,2008.0,nissan,altima,None,4 cylinders,...,fwd,None,None,None,https://images.craigslist.org/00V0V_an9fHTokkv...,"2008 NISSAN ALTIMA 2008 Nissan Altima S, 2....",None,co,39.0645,-108.5500
3,7082414500,https://corvallis.craigslist.org/ctd/d/portlan...,corvallis/albany,https://corvallis.craigslist.org,11000,2014.0,chevrolet,cruze,None,4 cylinders,...,fwd,None,sedan,silver,https://images.craigslist.org/00g0g_crAmR9fcFj...,NW Auto Locator Call/Text us at (508) 380-7260...,None,or,45.5303,-122.7030
4,7080508414,https://annarbor.craigslist.org/cto/d/fenton-1...,ann arbor,https://annarbor.craigslist.org,6000,1998.0,chevrolet,suburban,good,8 cylinders,...,4wd,full-size,truck,white,https://images.craigslist.org/00H0H_2WicFiQnQe...,"98 suburban 3/4 ton 4x4 454, auto 58,000 Origi...",None,mi,42.7851,-83.7294


In [15]:
df.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'vin', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long'],
      dtype='object')

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539759 entries, 0 to 539758
Data columns (total 25 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            539759 non-null  int64  
 1   url           539759 non-null  object 
 2   region        539759 non-null  object 
 3   region_url    539759 non-null  object 
 4   price         539759 non-null  int64  
 5   year          538772 non-null  float64
 6   manufacturer  516175 non-null  object 
 7   model         539740 non-null  object 
 8   condition     303707 non-null  object 
 9   cylinders     321264 non-null  object 
 10  fuel          536366 non-null  object 
 11  odometer      440783 non-null  float64
 12  title_status  536819 non-null  object 
 13  transmission  535786 non-null  object 
 14  vin           315400 non-null  object 
 15  drive         383987 non-null  object 
 16  size          168550 non-null  object 
 17  type          392290 non-null  object 
 18  pain

In [17]:
df.describe()

,id,price,year,odometer,lat,long
count,5.397590e+05,5.397590e+05,538772.000000,4.407830e+05,530785.000000,530785.000000
mean,7.087609e+09,1.954214e+05,2009.830982,1.008008e+05,38.432128,-94.040501
std,5.234176e+06,2.208252e+07,9.161334,1.767058e+05,5.916936,17.626658
min,7.065765e+09,0.000000e+00,0.000000,0.000000e+00,-84.911400,-176.810000
25%,7.084240e+09,4.400000e+03,2007.000000,4.703850e+04,34.197100,-106.782000
50%,7.088973e+09,9.500000e+03,2012.000000,9.330000e+04,39.077600,-88.100900
75%,7.092094e+09,1.792600e+04,2015.000000,1.377550e+05,42.449100,-81.219000
max,7.093537e+09,4.294967e+09,2021.000000,6.480922e+07,84.514800,161.016000


In [19]:
df.isnull().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year               987
manufacturer     23584
model               19
condition       236052
cylinders       218495
fuel              3393
odometer         98976
title_status      2940
transmission      3973
vin             224359
drive           155772
size            371209
type            147469
paint_color     174239
image_url           19
description         19
county          539759
state                0
lat               8974
long              8974
dtype: int64

In [20]:
df.shape

(539759, 25)

In [25]:
df.vin.value_counts()

WDZPE7CC9D5767661    262
77777777777777777     82
CALL 203 231 1817     76
Email for VIN         69
5TFTX4GN8CX009283     56
                    ... 
5TFCZ5AN6GX009118      1
4T1FA38P86U094197      1
5YFBURHEXGP400326      1
WDBTJ56H57F220874      1
KM8JUCAG4FU971683      1
Name: vin, Length: 181680, dtype: int64

In [26]:
df.drive.value_counts()

4wd    178592
fwd    133148
rwd     72247
Name: drive, dtype: int64

In [30]:
df['size'].value_counts()

full-size      92545
mid-size       47033
compact        25014
sub-compact     3958
Name: size, dtype: int64

#### Dealing with missing values

In [36]:
df = df.drop(columns = 'county')

In [40]:
df_raw = df.copy()

In [42]:
df.drop(columns = ['id', 'url', 'region_url', 'vin', 'image_url'], inplace = True)

In [47]:
df.isnull().sum()

region               0
price                0
year               987
manufacturer     23584
model               19
condition       236052
cylinders       218495
fuel              3393
odometer         98976
title_status      2940
transmission      3973
drive           155772
size            371209
type            147469
paint_color     174239
description         19
state                0
lat               8974
long              8974
dtype: int64

In [48]:
df.head()

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,description,state,lat,long
0,south dakota,20000,2013.0,gmc,acadia denali,excellent,None,gas,84300.0,clean,automatic,None,None,None,red,"84,300 miles New tires, well maintained. Love...",sd,43.3318,-99.8033
1,pueblo,5950,2016.0,kia,soul,None,None,gas,100784.0,None,automatic,fwd,None,SUV,black,"2016 KIA SOUL - $5,950 \tGallery Auto Auction ...",co,38.3536,-104.6230
2,western slope,4900,2008.0,nissan,altima,None,4 cylinders,gas,127755.0,clean,automatic,fwd,None,None,None,"2008 NISSAN ALTIMA 2008 Nissan Altima S, 2....",co,39.0645,-108.5500
3,corvallis/albany,11000,2014.0,chevrolet,cruze,None,4 cylinders,gas,73015.0,clean,automatic,fwd,None,sedan,silver,NW Auto Locator Call/Text us at (508) 380-7260...,or,45.5303,-122.7030
4,ann arbor,6000,1998.0,chevrolet,suburban,good,8 cylinders,gas,NaN,clean,automatic,4wd,full-size,truck,white,"98 suburban 3/4 ton 4x4 454, auto 58,000 Origi...",mi,42.7851,-83.7294


In [73]:
df.drop(columns = ['region', 'lat', 'long'], inplace = True)

In [74]:
df['type'].value_counts()

sedan          96119
SUV            94626
pickup         55934
truck          50419
coupe          21664
other          18351
hatchback      14791
wagon          12194
van            11006
convertible     9089
mini-van        6685
offroad          835
bus              577
Name: type, dtype: int64

In [80]:
df.shape

(539759, 17)

In [82]:
color_list = list(df.paint_color.unique())
color_list.remove(None)

In [83]:
color_list

['red',
 'black',
 'silver',
 'white',
 'blue',
 'grey',
 'brown',
 'yellow',
 'purple',
 'custom',
 'green',
 'orange']

In [84]:
df.head()

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,description,state,color
0,20000,2013.0,gmc,acadia denali,excellent,None,gas,84300.0,clean,automatic,None,None,None,red,"84,300 miles New tires, well maintained. Love...",sd,None
1,5950,2016.0,kia,soul,None,None,gas,100784.0,None,automatic,fwd,None,SUV,black,"2016 KIA SOUL - $5,950 \tGallery Auto Auction ...",co,black
2,4900,2008.0,nissan,altima,None,4 cylinders,gas,127755.0,clean,automatic,fwd,None,None,None,"2008 NISSAN ALTIMA 2008 Nissan Altima S, 2....",co,None
3,11000,2014.0,chevrolet,cruze,None,4 cylinders,gas,73015.0,clean,automatic,fwd,None,sedan,silver,NW Auto Locator Call/Text us at (508) 380-7260...,or,black
4,6000,1998.0,chevrolet,suburban,good,8 cylinders,gas,NaN,clean,automatic,4wd,full-size,truck,white,"98 suburban 3/4 ton 4x4 454, auto 58,000 Origi...",mi,None


In [86]:
df.iloc[2, 14]

'2008 NISSAN ALTIMA    2008 Nissan Altima S, 2.5LT, 4CYL, Auto, FWD. Sharp car with power seat, Push button start, keyless entry, tinted windows, steering wheel controls, cruise control, low miles, folding rear seats, great on gas with lots of room. Come check this low mile Altima out.  Works great for first time buyers!       Vehicle Information  Make: NISSAN Model: ALTIMA Trim: S Exterior Color: GRAY Engine: 2.5L Fuel: Gasoline Transmission: Continuously Variable Drive: FWD Miles: 127,755 VIN: 1N4AL21E08N425305 Stock #: 0159    Dealership Info  LIGHTHOUSE AUTO SALES LLC 1346 PITKIN AVE GRAND JUNCTION, CO 81501  TEXT: 970-852-2445 PHONE: 970-852-2445  Price:   $4,900.00 + $100 DH= $5,000 Total asking price             LIGHTHOUSE AUTO SALES LLC   WEBSITE: lighthousedealership.com 2008 NISSAN ALTIMA - Stock # 0159 Offered by: LIGHTHOUSE AUTO SALES LLC 68259-0159-79889-6532791'

In [69]:
def check_for_color(x):
    if x is not None:
        for i in color_list:
            if i in x.lower().split():
                print(x)
                return i

In [70]:
df['color'] = df['description'].apply(check_for_color)

In [71]:
df.color.value_counts()

black     124280
white      38135
red        28313
silver     22167
blue       20774
custom      6902
green       4063
grey        3310
brown       2730
orange      1743
yellow       566
purple       212
Name: color, dtype: int64

In [89]:
df.drop('paint_color_v2', axis = 1, inplace = True)

In [95]:
df['color'] = np.where(df['paint_color']== None, df['color'], df['paint_color'])

In [97]:
df.isnull().sum()

price                0
year               987
manufacturer     23584
model               19
condition       236052
cylinders       218495
fuel              3393
odometer         98976
title_status      2940
transmission      3973
drive           155772
size            371209
type            147469
paint_color     174239
description         19
state                0
color           174239
dtype: int64

In [98]:
df_before_model = df.copy()

In [139]:
# df = df_before_model.copy()

In [140]:
df.drop(columns = ['model', 'color', 'description'], inplace = True)
df = df.dropna()

In [141]:
df.shape

(107433, 14)

In [142]:
df.head()

,price,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
6,10500,2002.0,ford,fair,8 cylinders,gas,189910.0,clean,automatic,4wd,full-size,truck,white,wv
9,0,2011.0,cadillac,excellent,8 cylinders,gas,58988.0,clean,automatic,fwd,full-size,sedan,red,sd
14,7995,2006.0,chevrolet,good,8 cylinders,gas,99920.0,clean,automatic,fwd,full-size,sedan,silver,mo
17,9995,2014.0,ford,excellent,6 cylinders,gas,70000.0,clean,automatic,fwd,full-size,sedan,red,ny
21,8700,2013.0,chevrolet,good,4 cylinders,gas,500000.0,clean,automatic,fwd,mid-size,sedan,blue,wv


In [143]:
def cylinder_transformation(x):
    if x != 'other':
        return int(x.split()[0])
    elif x == 'other':
        return 'other'

df['cylinders'] = df['cylinders'].apply(cylinder_transformation)

In [145]:
df = pd.get_dummies(df)

In [146]:
df.shape

(107433, 157)

In [147]:
df.head()

,price,year,odometer,manufacturer_acura,manufacturer_alfa-romeo,manufacturer_aston-martin,manufacturer_audi,manufacturer_bmw,manufacturer_buick,manufacturer_cadillac,...,state_sd,state_tn,state_tx,state_ut,state_va,state_vt,state_wa,state_wi,state_wv,state_wy
6,10500,2002.0,189910.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,0,2011.0,58988.0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
14,7995,2006.0,99920.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,9995,2014.0,70000.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21,8700,2013.0,500000.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


##### Model training 

In [148]:
y = df['price']
X = df.drop('price', axis = 1)

In [149]:
X.shape, y.shape

((107433, 156), (107433,))

In [150]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [151]:
skf5 = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)

from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

linreg.fit(X_train, y_train)

from sklearn.metrics import mean_absolute_error
# Evaluation on test dataset
y_pred_linreg = linreg.predict(X_test)
print(mean_absolute_error(y_test, y_pred_linreg))

563298.4007467956


In [152]:
### tunning

In [153]:
from sklearn.linear_model import Lasso

parameters = {'alpha': [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 1]}
lassoreg = Lasso(normalize=True)
lassoreg_gcv = GridSearchCV(lassoreg, parameters, n_jobs = -1, cv = skf5, verbose = 1, 
                    scoring = 'neg_mean_absolute_error')
lassoreg_gcv.fit(X_train, y_train)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


/Users/marichka/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:  5.9min finished
/Users/marichka/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
       error_score='raise-deprecating',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'alpha': [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=1)

In [156]:
lassoreg_gcv.best_score_

-560899.3307720867

In [157]:
lassoreg_gcv.best_params_

{'alpha': 1}

In [158]:
# Initialize our classifier with default parameters
lassoreg = Lasso(normalize=True, alpha = 1)

# Evaluation on test dataset
y_pred_lassoreg = lassoreg.fit(X_train, y_train).predict(X_test)

In [160]:
# save the model to disk
import pickle
filename = 'lassoreg.sav'
pickle.dump(lassoreg, open(filename, 'wb'))

#### put model to google cloud

In [171]:
client = storage.Client()
bucket = client.get_bucket('car_platform_gcs_bucket')
blob = bucket.blob('models/lassoreg.sav')
blob.upload_from_filename('lassoreg.sav')